# Use SUPER_ANNU_Template customized notebook template

In [1]:
#Snowpark lib
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

# Sklearn Libraries
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import DBSCAN

np.random.seed(0)

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [32]:
import pickle
with open('Churn_Classifier.pkl','rb') as f:  ## use rb while reading the fie
    churn_model = pickle.load(f)

In [33]:
table_name = 'MEMBER_FUNDS_ENRICHED_DETAILS'
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [36]:
df = df.drop(["MEMBER_DOB_DT", 'ALLOCATION_DT', "CHURN_DT", "RETIREMENT_DT"], axis =1 )

In [37]:
frame = df[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()

In [38]:
X = frame.drop("CHURN_FLAG", axis=1)
y = frame["CHURN_FLAG"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [39]:
def score(model, request):
    payload_dict = eval(request.json["payload"])
    data = pd.DataFrame.from_dict(payload_dict)
    frame = data[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED']].copy()
    result = model.predict(frame)
    prediction = pd.DataFrame({"CHURN_PREDICTION":result})
    prediction["CHURN_PREDICTION"] = prediction["CHURN_PREDICTION"].apply(lambda x: "Y" if x == 1 else "N")
    probability = model.predict_proba(frame)
    prediction["Probability"] = [round(k[1],4) for k in probability]    
    prediction = prediction.to_dict()
    return pd.DataFrame(prediction)

In [40]:
import requests

In [41]:
payload = df.head(10).to_dict()
req = requests.Request()
req.json = {"payload":str(payload)}
y = req
yo = score(clf, y)
yo

,CHURN_PREDICTION,Probability
0,Y,0.8526
1,Y,0.9883
2,Y,0.5670
3,Y,0.7961
4,Y,0.8032
5,Y,0.5919
6,Y,0.8782
7,Y,0.5755
8,Y,0.9691
9,Y,0.9606


In [42]:
y_pred = yo["CHURN_PREDICTION"]
y_prob = yo["Probability"]

In [43]:
y_train = y_train.to_frame()
y_test = y_test.to_frame()
y_pred = y_pred.to_frame()
y_prob = y_prob.to_frame()

In [44]:
type(X_train), type(X_test),type(y_train), type(y_test), type(y_pred)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [46]:
register_model(
    model_obj=churn_model, 
    session=my_session,
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    y_pred=y_pred,
    y_prob = y_prob,
    source="Notebook",
    dataset_name="MEMBER_FUNDS_ENRICHED_DETAILS",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="Churn_Classifier_RF_Model",
    description="PREDICTING_FUND_CHURN",
    flavour="sklearn",
    model_type="classification",
    conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%
(0000) pos_label=1 is not a valid label. It must be one of ['N', 'Y']
(0000) pos_label=1 is not a valid label. It must be one of ['N', 'Y']
(0000) pos_label=1 is not a valid label. It must be one of ['N', 'Y']


(1300) (1304): 01b6c307-070e-1713-0072-f30310493d6e: 100038 (22018): Numeric value 'N' is not recognized
Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%


Error in while calculating roc_auc 
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Error in while calculating feature_importance 
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_1BCAA7D6_FE9E_4FB7_AF4D_7C27E92AD7B4_FDC_CHURN_CLASSIFIER_RF_MODEL' registered successfully."